In [1]:
from tqdm import tqdm
import regex as re
import os
import pandas as pd

In [193]:
print("Started scanning...")
result_files = {}
for root, dirs, files in os.walk("/home/maximilian/Master/pre_ablations/"):
    for dir in dirs:
        eval_reps = os.path.join(root, dir)
        if re.findall(".*_.*", dir):
            for rep_root, rep_dirs, rep_files in os.walk(eval_reps):
                for rep_dir in rep_dirs:
                    file_lst=[]
                    rep_path = os.path.join(eval_reps, rep_dir)
                    if rep_dir == "results":
                        for filename in tqdm(os.listdir(rep_path)):
                            file_lst.append(os.path.join(rep_path, filename))
                        result_files[dir] = file_lst

Started scanning...


100%|██████████| 130/130 [00:00<00:00, 536672.76it/s]


In [194]:
weathers = {"train": [1, 6, 10, 14], "test": [3,8]}
def weather_mapping(value):
    for key, value_lst in weathers.items():
        if value in value_lst:
            return key

In [195]:
def get_ablations_dict():
    return {"bev":0, "detectboxes": 0, "speed":0, "prevnum":0, "backbone":0, "datarep": 2, "augment": 0}

In [196]:
df_lst=[]
for eval_rep, path_lst in tqdm(result_files.items()):
    for path in path_lst:
        eval_results = pd.read_csv(path)
        df_lst.append(eval_results)
df = pd.concat(df_lst, ignore_index=True)
df["success"] = (df["timeout_blocked"] == 0) & (df["collision"] == 0)
df["success"] = df["success"].astype("int")
df["weather"] = df["weather"].map(lambda x: weather_mapping(x))
ablations_dict=get_ablations_dict()

100%|██████████| 3/3 [00:00<00:00,  4.98it/s]


In [200]:
#watch out when considering multiple training reps
groups = df.groupby(["baseline", "town","traffic", "weather", "eval_rep", "training_rep",*ablations_dict]).agg(
    timeouts_percentage=("timeout_blocked", "mean"),
    collisions_percentage=("collision", "mean"),
    success_percentage=("success", "mean"),
)
groups = groups.groupby(["baseline", "town", "traffic", "weather",*ablations_dict]).agg(
    timeout_mean=("timeouts_percentage", "mean"),
    timeout_std=("timeouts_percentage", "std"),
    collisions_mean=("collisions_percentage", "mean"),
    collisions_std=("collisions_percentage", "std"),
    success_mean=("success_percentage", "mean"),
    success_std=("success_percentage", "std"),
)

In [201]:
groups=(groups*100).round(2)

In [82]:
pd.set_option('display.max_rows', 500)

In [202]:
groups

timeout_mean  \
baseline town   traffic weather bev detectboxes speed prevnum backbone  datarep augment                 
arp      Town01 1       train   0   0           0     0       unrolling 1       0                5.33   
                2       train   0   0           0     0       unrolling 1       0                6.67   
         Town02 1       train   0   0           0     0       unrolling 1       0                0.00   
                2       train   0   0           0     0       unrolling 1       0               13.33   
bcoh     Town01 1       train   0   0           0     0       unrolling 1       0                4.00   
                2       train   0   0           0     0       unrolling 1       0                4.00   
         Town02 1       train   0   0           0     0       unrolling 1       0                4.00   
                2       train   0   0           0     0       unrolling 1       0                4.00   
bcso     Town01 1       train   0   0           0     0       unrolling 1       0                0.00   
                                                                        3       0                8.00   
                                1   1           0     0       unrolling 1       0                5.33   
                2       train   0   0           0     0       unrolling 1       0                4.00   
                                                                        3       0                4.00   
                                1   1           0     0       unrolling 1       0                1.33   
         Town02 1       train   0   0           0     0       unrolling 1       0                9.33   
                                                                        3       0               13.33   
                                1   1           0     0       unrolling 1       0                0.00   
                2       train   0   0           0     0       unrolling 1       0               14.67   
                                                                        3       0               17.33   
                                1   1           0     0       unrolling 1       0                5.33   

                                                                                         timeout_std  \
baseline town   traffic weather bev detectboxes speed prevnum backbone  datarep augment                
arp      Town01 1       train   0   0           0     0       unrolling 1       0               6.11   
                2       train   0   0           0     0       unrolling 1       0               2.31   
         Town02 1       train   0   0           0     0       unrolling 1       0               0.00   
                2       train   0   0           0     0       unrolling 1       0               4.62   
bcoh     Town01 1       train   0   0           0     0       unrolling 1       0               0.00   
                2       train   0   0           0     0       unrolling 1       0               4.00   
         Town02 1       train   0   0           0     0       unrolling 1       0               4.00   
                2       train   0   0           0     0       unrolling 1       0               6.93   
bcso     Town01 1       train   0   0           0     0       unrolling 1       0               0.00   
                                                                        3       0               0.00   
                                1   1           0     0       unrolling 1       0               4.62   
                2       train   0   0           0     0       unrolling 1       0               4.00   
                                                                        3       0               0.00   
                                1   1           0     0       unrolling 1       0               2.31   
         Town02 1       train   0   0           0     0       unrolling 1       0               2.31   
          

__Traffic: Regular,\
Town: Training Town__

In [225]:
groups.loc[(slice(None), "Town01", 1, "train")].reset_index(level=[6,7], drop=True)

timeout_mean  \
baseline town   traffic weather bev detectboxes backbone  datarep augment                 
arp      Town01 1       train   0   0           unrolling 1       0                5.33   
bcoh     Town01 1       train   0   0           unrolling 1       0                4.00   
bcso     Town01 1       train   0   0           unrolling 1       0                0.00   
                                                          3       0                8.00   
                                1   1           unrolling 1       0                5.33   

                                                                           timeout_std  \
baseline town   traffic weather bev detectboxes backbone  datarep augment                
arp      Town01 1       train   0   0           unrolling 1       0               6.11   
bcoh     Town01 1       train   0   0           unrolling 1       0               0.00   
bcso     Town01 1       train   0   0           unrolling 1       0               0.00   
                                                          3       0               0.00   
                                1   1           unrolling 1       0               4.62   

                                                                           collisions_mean  \
baseline town   traffic weather bev detectboxes backbone  datarep augment                    
arp      Town01 1       train   0   0           unrolling 1       0                  21.33   
bcoh     Town01 1       train   0   0           unrolling 1       0                  36.00   
bcso     Town01 1       train   0   0           unrolling 1       0                   4.00   
                                                          3       0                   1.33   
                                1   1           unrolling 1       0                   1.33   

                                                                           collisions_std  \
baseline town   traffic weather bev detectboxes backbone  datarep augment                   
arp      Town01 1       train   0   0           unrolling 1       0                 14.05   
bcoh     Town01 1       train   0   0           unrolling 1       0                  4.00   
bcso     Town01 1       train   0   0           unrolling 1       0                  4.00   
                                                          3       0                  2.31   
                                1   1           unrolling 1       0                  2.31   

                                                                           success_mean  \
baseline town   traffic weather bev detectboxes backbone  datarep augment                 
arp      Town01 1       train   0   0           unrolling 1       0               73.33   
bcoh     Town01 1       train   0   0           unrolling 1       0               60.00   
bcso     Town01 1       train   0   0           unrolling 1       0               96.00   
                                                          3       0               90.67   
                                1   1           unrolling 1       0               93.33   

                                                                           success_std  
baseline town   traffic weather bev detectboxes backbone  datarep augment               
arp      Town01 1       train   0   0           unrolling 1       0               8.33  
bcoh     Town01 1       train   0   0           unrolling 1       0               4.00  
bcso     Town01 1       train   0   0           unrolling 1       0               4.00  
                                                          3       0               2.31  
                                1   1           unrolling 1       0               6.11

__Traffic: Dense,\
Town: Training Town__

In [226]:
groups.loc[(slice(None), "Town01", 2, "train")].reset_index(level=[6,7], drop=True)

timeout_mean  \
baseline town   traffic weather bev detectboxes backbone  datarep augment                 
arp      Town01 2       train   0   0           unrolling 1       0                6.67   
bcoh     Town01 2       train   0   0           unrolling 1       0                4.00   
bcso     Town01 2       train   0   0           unrolling 1       0                4.00   
                                                          3       0                4.00   
                                1   1           unrolling 1       0                1.33   

                                                                           timeout_std  \
baseline town   traffic weather bev detectboxes backbone  datarep augment                
arp      Town01 2       train   0   0           unrolling 1       0               2.31   
bcoh     Town01 2       train   0   0           unrolling 1       0               4.00   
bcso     Town01 2       train   0   0           unrolling 1       0               4.00   
                                                          3       0               0.00   
                                1   1           unrolling 1       0               2.31   

                                                                           collisions_mean  \
baseline town   traffic weather bev detectboxes backbone  datarep augment                    
arp      Town01 2       train   0   0           unrolling 1       0                  28.00   
bcoh     Town01 2       train   0   0           unrolling 1       0                  42.67   
bcso     Town01 2       train   0   0           unrolling 1       0                  10.67   
                                                          3       0                   6.67   
                                1   1           unrolling 1       0                   1.33   

                                                                           collisions_std  \
baseline town   traffic weather bev detectboxes backbone  datarep augment                   
arp      Town01 2       train   0   0           unrolling 1       0                 10.58   
bcoh     Town01 2       train   0   0           unrolling 1       0                  6.11   
bcso     Town01 2       train   0   0           unrolling 1       0                  4.62   
                                                          3       0                  2.31   
                                1   1           unrolling 1       0                  2.31   

                                                                           success_mean  \
baseline town   traffic weather bev detectboxes backbone  datarep augment                 
arp      Town01 2       train   0   0           unrolling 1       0               65.33   
bcoh     Town01 2       train   0   0           unrolling 1       0               53.33   
bcso     Town01 2       train   0   0           unrolling 1       0               85.33   
                                                          3       0               89.33   
                                1   1           unrolling 1       0               97.33   

                                                                           success_std  
baseline town   traffic weather bev detectboxes backbone  datarep augment               
arp      Town01 2       train   0   0           unrolling 1       0               8.33  
bcoh     Town01 2       train   0   0           unrolling 1       0               2.31  
bcso     Town01 2       train   0   0           unrolling 1       0               2.31  
                                                          3       0               2.31  
                                1   1           unrolling 1       0               4.62

__Traffic: Regular,\
Town: Test Town__

In [222]:
groups.loc[(slice(None), "Town02", 1, "train")].reset_index(level=[6,7], drop=True)

timeout_mean  \
baseline town   traffic weather bev detectboxes backbone  datarep augment                 
arp      Town02 1       train   0   0           unrolling 1       0                0.00   
bcoh     Town02 1       train   0   0           unrolling 1       0                4.00   
bcso     Town02 1       train   0   0           unrolling 1       0                9.33   
                                                          3       0               13.33   
                                1   1           unrolling 1       0                0.00   

                                                                           timeout_std  \
baseline town   traffic weather bev detectboxes backbone  datarep augment                
arp      Town02 1       train   0   0           unrolling 1       0               0.00   
bcoh     Town02 1       train   0   0           unrolling 1       0               4.00   
bcso     Town02 1       train   0   0           unrolling 1       0               2.31   
                                                          3       0               2.31   
                                1   1           unrolling 1       0               0.00   

                                                                           collisions_mean  \
baseline town   traffic weather bev detectboxes backbone  datarep augment                    
arp      Town02 1       train   0   0           unrolling 1       0                  26.67   
bcoh     Town02 1       train   0   0           unrolling 1       0                  32.00   
bcso     Town02 1       train   0   0           unrolling 1       0                  10.67   
                                                          3       0                  33.33   
                                1   1           unrolling 1       0                  21.33   

                                                                           collisions_std  \
baseline town   traffic weather bev detectboxes backbone  datarep augment                   
arp      Town02 1       train   0   0           unrolling 1       0                  2.31   
bcoh     Town02 1       train   0   0           unrolling 1       0                  6.93   
bcso     Town02 1       train   0   0           unrolling 1       0                  2.31   
                                                          3       0                 10.07   
                                1   1           unrolling 1       0                  6.11   

                                                                           success_mean  \
baseline town   traffic weather bev detectboxes backbone  datarep augment                 
arp      Town02 1       train   0   0           unrolling 1       0               73.33   
bcoh     Town02 1       train   0   0           unrolling 1       0               64.00   
bcso     Town02 1       train   0   0           unrolling 1       0               80.00   
                                                          3       0               53.33   
                                1   1           unrolling 1       0               78.67   

                                                                           success_std  
baseline town   traffic weather bev detectboxes backbone  datarep augment               
arp      Town02 1       train   0   0           unrolling 1       0               2.31  
bcoh     Town02 1       train   0   0           unrolling 1       0               4.00  
bcso     Town02 1       train   0   0           unrolling 1       0               4.00  
                                                          3       0               8.33  
                                1   1           unrolling 1       0               6.11

__Traffic: Dense,\
Town: Test Town__

In [220]:
groups.loc[(slice(None), "Town02", 2, "train")].reset_index(level=[6,7], drop=True)

timeout_mean  \
baseline town   traffic weather bev detectboxes backbone  datarep augment                 
arp      Town02 2       train   0   0           unrolling 1       0               13.33   
bcoh     Town02 2       train   0   0           unrolling 1       0                4.00   
bcso     Town02 2       train   0   0           unrolling 1       0               14.67   
                                                          3       0               17.33   
                                1   1           unrolling 1       0                5.33   

                                                                           timeout_std  \
baseline town   traffic weather bev detectboxes backbone  datarep augment                
arp      Town02 2       train   0   0           unrolling 1       0               4.62   
bcoh     Town02 2       train   0   0           unrolling 1       0               6.93   
bcso     Town02 2       train   0   0           unrolling 1       0               9.24   
                                                          3       0               2.31   
                                1   1           unrolling 1       0               6.11   

                                                                           collisions_mean  \
baseline town   traffic weather bev detectboxes backbone  datarep augment                    
arp      Town02 2       train   0   0           unrolling 1       0                  34.67   
bcoh     Town02 2       train   0   0           unrolling 1       0                  61.33   
bcso     Town02 2       train   0   0           unrolling 1       0                  26.67   
                                                          3       0                  34.67   
                                1   1           unrolling 1       0                  25.33   

                                                                           collisions_std  \
baseline town   traffic weather bev detectboxes backbone  datarep augment                   
arp      Town02 2       train   0   0           unrolling 1       0                  8.33   
bcoh     Town02 2       train   0   0           unrolling 1       0                 10.07   
bcso     Town02 2       train   0   0           unrolling 1       0                  4.62   
                                                          3       0                  2.31   
                                1   1           unrolling 1       0                  2.31   

                                                                           success_mean  \
baseline town   traffic weather bev detectboxes backbone  datarep augment                 
arp      Town02 2       train   0   0           unrolling 1       0               52.00   
bcoh     Town02 2       train   0   0           unrolling 1       0               34.67   
bcso     Town02 2       train   0   0           unrolling 1       0               58.67   
                                                          3       0               48.00   
                                1   1           unrolling 1       0               69.33   

                                                                           success_std  
baseline town   traffic weather bev detectboxes backbone  datarep augment               
arp      Town02 2       train   0   0           unrolling 1       0              10.58  
bcoh     Town02 2       train   0   0           unrolling 1       0              11.55  
bcso     Town02 2       train   0   0           unrolling 1       0              12.22  
                                                          3       0               4.00  
                                1   1           unrolling 1       0               4.62

In [24]:
#groups.to_csv(os.path.join("/home/maximilian/Master/first_large_evaluation_all_settings_all_baselines_partially_wrong_speed_input", os.path.join("/home/maximilian/Master/first_large_evaluation_all_settings_all_baselines_partially_wrong_speed_input","combined.csv")))